In [ ]:
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.vgg16 import VGG16
from keras.metrics import FalseNegatives
from keras.metrics import FalsePositives
from keras.preprocessing.image import ImageDataGenerator
import os
import zipfile
import gzip
import PIL
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image
from numpy import asarray
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import AveragePooling2D
from keras.layers import Flatten
from keras.layers import BatchNormalization
import tensorflow as tf
from tensorflow.python.framework import ops
import matplotlib.pyplot as plt
from keras.utils import plot_model
# tf.compat.v1.disable_eager_execution()
from sklearn import linear_model
import pandas as pd

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
zip = zipfile.ZipFile("/content/gdrive/My Drive/covid_dataset/covid19-detection-xray-dataset.zip","r")
unzipped=zip.extractall()

In [ ]:
train_orig_X=[]
train_orig_Y=[]

In [ ]:
def append_to_trainin_set(directory,y):
  for filename in os.listdir(str(directory)):
    image = Image.open(directory+"/"+filename)
    image=image.resize((224,224))
    data = asarray(image)
    train_orig_X.append(data)
    train_orig_Y.append(y)

In [ ]:
append_to_trainin_set("/content/NonAugmentedTrain/BacterialPneumonia",0)
print(len(train_orig_X))
append_to_trainin_set("/content/NonAugmentedTrain/COVID-19",1)
print(len(train_orig_X))
append_to_trainin_set("/content/NonAugmentedTrain/Normal",0)
print(len(train_orig_X))
append_to_trainin_set("/content/NonAugmentedTrain/ViralPneumonia",0)
print(len(train_orig_X))
append_to_trainin_set("/content/ValData/BacterialPneumonia",0)
print(len(train_orig_X))
append_to_trainin_set("/content/ValData/COVID-19",1)
print(len(train_orig_X))
append_to_trainin_set("/content/ValData/Normal",0)
print(len(train_orig_X))
append_to_trainin_set("/content/ValData/ViralPneumonia",0)
print(len(train_orig_X))

650
710
1590
2002
2326
2335
2785
2990


In [ ]:
print(len(train_orig_X))
i=0
for image in train_orig_X:
  train_orig_X[i]=np.resize(image,(224,224,3))
  i+=1

2990


In [ ]:
train_orig_X=np.array(train_orig_X)
train_orig_X.resize(2990,224,224,3)

In [ ]:
skf = StratifiedKFold(n_splits=10)
train_orig_Y=np.array(train_orig_Y)
skf.get_n_splits(train_orig_X, (train_orig_Y))
for train_index, test_index in skf.split(train_orig_X, train_orig_Y):
  train_X, test_X = train_orig_X[train_index], train_orig_X[test_index]
  train_Y, test_Y = train_orig_Y[train_index], train_orig_Y[test_index]
  model=Sequential()
  model.add(Conv2D(32,(3,3),strides=(1,1),name="first",activation='relu',input_shape=(224,224,3)))
  model.add(BatchNormalization())
  model.add(AveragePooling2D((2,2),strides=(2,2)))
  model.add(Conv2D(64,(3,3),strides=(1,1),activation='relu'))
  model.add(BatchNormalization())
  model.add(AveragePooling2D((2,2),strides=(2,2)))
  model.add(Conv2D(128,(3,3),strides=(1,1),activation='relu'))
  model.add(BatchNormalization())
  model.add(AveragePooling2D((2,2),strides=(2,2)))
  model.add(Flatten())
  model.add(Dense(units=512,activation='relu'))
  model.add(Dense(units=128,activation='relu'))
  model.add(Dense(units=1,activation='sigmoid'))
  model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])
  model.fit(train_X,(train_Y),batch_size=16,epochs=40,verbose=1,class_weight={0:1,1:65})
  print(model.evaluate(test_X,test_Y))
  pred=model.predict(test_X)
  pred=pred>=0.5
  print(classification_report(test_Y,pred))

Epoch 1/40
2691/2691 [==============================] - 34s 13ms/step - loss: 53.5473 - accuracy: 0.7960
Epoch 2/40
2691/2691 [==============================] - 27s 10ms/step - loss: 27.6433 - accuracy: 0.9056
Epoch 3/40
2691/2691 [==============================] - 27s 10ms/step - loss: 14.9436 - accuracy: 0.9104
Epoch 4/40
2691/2691 [==============================] - 27s 10ms/step - loss: 15.5764 - accuracy: 0.9391
Epoch 5/40
2691/2691 [==============================] - 27s 10ms/step - loss: 33.1074 - accuracy: 0.9164
Epoch 6/40
2691/2691 [==============================] - 27s 10ms/step - loss: 7.2647 - accuracy: 0.9569
Epoch 7/40
2691/2691 [==============================] - 27s 10ms/step - loss: 9.3697 - accuracy: 0.9443
Epoch 8/40
2691/2691 [==============================] - 27s 10ms/step - loss: 10.0874 - accuracy: 0.9565
Epoch 9/40
2691/2691 [==============================] - 27s 10ms/step - loss: 2.2728 - accuracy: 0.9721
Epoch 10/40
2691/2691 [==============================] - 2